## 컨텐츠 기반 추천시스템 (Contents Based Recommendation System)
- 컨텐츠 기반은 사용자가 이전에 구매한 상품중에 좋아하는 것과 유사한 상품들을 추천해준다.
- items(1~N) -> 벡터로 변환 -> 유사한 벡터들간 유사도 계산 -> 자신과 유사한 벡터 추출


### 유사도 함수 : 문서간 유사도를 계산 

#### 1. 유클리디안 유사도 
>- 유클리디안 유사도 = 1 / (유클리디안 거리 + 1e-05)
>- 장점 : 계산 쉬움
>- 단점 : p와 q의 분포가 다르거나 범위가 다르면 상관성을 놓침

#### 2. 코사인 유사도
>- 장점 : 벡터의 크기가 중요하지 않은 경우 거리를 측정하기 위한 메트릭으로 사용 (예 : 문서>내에서 단어 빈도수. 문서의 길이와 상관없이 단어의 비율만 확인)
>- 단점 : 벡터의 크기가 중요한 경우는 작동 어려움 

-> 유클리디안은 가까이 있는 벡터를 유사하다고 하지만 코사인은 비슷한 각도(세타)값을 갖는 벡터를 유사하다고 한다. 코사인유사도는 비슷한 선상에 있는 벡터를 선택함 
따라서 두 유사도는 다른 결과를 낼 수 있다. 상황에 따른 선택 필요
### TF-IDF 
> TF * IDF = TF-IDF(d,t)
>- TF(d,t) : 특정 문서 d 에서 특정 단어 t의 등장 횟수
>- DF(t) : 특정 단어 t가 등장한 문서의 수 
>- IDF(d,t) : DF(t)에 반비하는 수 log(n/1+df(t))

>- 장점
>: 직관적 해석 가능
>- 단점
>: 대규모 말뭉치 다룰 때 메모리상의 문제 발생.
  높은 차원 가짐.
  매우 sparse 한 형태의 데이터임.


In [7]:
docs = [
    '먹고 싶은 사과', # 문서0
    '먹고 싶은 바나나', #문서1 
    '길고 노란 바나나 바나나', #문서2
    '저는 과일이 좋아요' #문서3
]
docs

['먹고 싶은 사과', '먹고 싶은 바나나', '길고 노란 바나나 바나나', '저는 과일이 좋아요']

#### CounterVectorizer

CounterVectorizer는 TF까지만 거친 vectorizer이다.

빈도수를 기반으로 많이 나오는 중요한 단어들을 잡아준다.

Counter Vectorizer는 단순 빈도만을 계산하기에 조사, 관사처럼 의미는 없지만 문장에 많이 등장하 는 단어들도 높게 쳐주는 한계가 있다.이러한 단어들에는 패널티를 줘서 적절하게 중요한 단어만을 잡아내는 게 TF-IDF 기법이다.

In [8]:
# CounterVectorizer 객체 생성
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()

In [10]:
# 문장(item)을 CounterVectorizer 형태로 변형
countvect = vect.fit_transform(docs)
countvect # 4x9 : 4개의 문서에 9개의 단어

<4x9 sparse matrix of type '<class 'numpy.int64'>'
	with 12 stored elements in Compressed Sparse Row format>

In [12]:
# toarray()를 통해서 문장이 Vector 형태의 값을 얻을 수 있음
# 하지만 각 인덱스와 컬럼이 무엇을 의미하는지에 대해서는 알 수 없음
countvect.toarray()

array([[0, 0, 0, 1, 0, 1, 1, 0, 0],
       [0, 0, 0, 1, 1, 0, 1, 0, 0],
       [0, 1, 1, 0, 2, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 1, 1]])

In [13]:
# Vector의 컬럼을 딕셔너리 형태로 확인 가능
vect.vocabulary_

{'먹고': 3,
 '싶은': 6,
 '사과': 5,
 '바나나': 4,
 '길고': 1,
 '노란': 2,
 '저는': 7,
 '과일이': 0,
 '좋아요': 8}

In [15]:
# 위에서 정렬되지 않은 Vector의 컬럼을 정렬한다
sorted(vect.vocabulary_)

['과일이', '길고', '노란', '먹고', '바나나', '사과', '싶은', '저는', '좋아요']

In [21]:
# array를 DataFrame 으로 만들기
import pandas as pd
countvect_df = pd.DataFrame(countvect.toarray(), columns = sorted(vect.vocabulary_))
countvect_df.index = ['문서1', '문서2', '문서3', '문서4'] #인덱스 설정(안하면 0,1,2,3으로 나옴)
countvect_df

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
문서1,0,0,0,1,0,1,1,0,0
문서2,0,0,0,1,1,0,1,0,0
문서3,0,1,1,0,2,0,0,0,0
문서4,1,0,0,0,0,0,0,1,1


In [22]:
# 위의 DataFrame 형태의 유사도를 계산 
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(countvect_df, countvect_df)

array([[1.        , 0.66666667, 0.        , 0.        ],
       [0.66666667, 1.        , 0.47140452, 0.        ],
       [0.        , 0.47140452, 1.        , 0.        ],
       [0.        , 0.        , 0.        , 1.        ]])

문서1은 문서2와 유사하다는 결론을 얻을 수 있다. 동일한 방식으로 TF-IDF를 수행해보자.

In [29]:
# 문서(item)을 벡터로 변환하는 과정
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer() # TfidfVectorizer 객체 생성
tfvect = vect.fit_transform(docs).toarray() # 문장을 TfidfVectorizer 형태로 변형하고 array로 반환
tfvect

array([[0.        , 0.        , 0.        , 0.52640543, 0.        ,
        0.66767854, 0.52640543, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.57735027, 0.57735027,
        0.        , 0.57735027, 0.        , 0.        ],
       [0.        , 0.47212003, 0.47212003, 0.        , 0.7444497 ,
        0.        , 0.        , 0.        , 0.        ],
       [0.57735027, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.57735027, 0.57735027]])

In [31]:
tfidv_df = pd.DataFrame(tfvect, columns=sorted(vect.vocabulary_))
tfidv_df.index = ['문서1', '문서2', '문서3', '문서4']
tfidv_df

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
문서1,0.00000,0.00000,0.00000,0.526405,0.00000,0.667679,0.526405,0.00000,0.00000
문서2,0.00000,0.00000,0.00000,0.577350,0.57735,0.000000,0.577350,0.00000,0.00000
문서3,0.00000,0.47212,0.47212,0.000000,0.74445,0.000000,0.000000,0.00000,0.00000
문서4,0.57735,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000,0.57735,0.57735


In [32]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(tfidv_df, tfidv_df)

array([[1.        , 0.60784064, 0.        , 0.        ],
       [0.60784064, 1.        , 0.42980824, 0.        ],
       [0.        , 0.42980824, 1.        , 0.        ],
       [0.        , 0.        , 0.        , 1.        ]])

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(max_features=4)
tfvect = vect.fit_transform(docs)
tfvect

<4x4 sparse matrix of type '<class 'numpy.float64'>'
	with 7 stored elements in Compressed Sparse Row format>

In [35]:
tfidv_df = pd.DataFrame(tfvect.toarray(), columns=sorted(vect.vocabulary_))
tfidv_df.index = ['문서1', '문서2', '문서3', '문서4']
tfidv_df

,과일이,먹고,바나나,싶은
문서1,0.0,0.707107,0.00000,0.707107
문서2,0.0,0.577350,0.57735,0.577350
문서3,0.0,0.000000,1.00000,0.000000
문서4,1.0,0.000000,0.00000,0.000000
